# 信号処理用ノートブック

## ライブラリのインポート

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

from lib import SignalProcessor, Util

## 設定ファイルの読み込み

In [ ]:
# 設定ファイルの読み込み
with open(f"{Util.get_root_dir()}/../config/config.json", "r") as f:
    config = json.load(f)

## 信号処理パイプライン

In [ ]:
# 共通ファイルを取得
common_file = Util.get_common_files(path_list=[f"{Util.get_root_dir()}/../data/csv-data/{field_device}/amp/" for field_device in config["FieldDevice"]["Pcap"]])

In [ ]:
for field_device in sorted(config["FieldDevice"]["Pcap"]):
    for file_name in tqdm(common_file):
        # ファイルのパスを取得
        file_path = f"{Util.get_root_dir()}/../data/csv-data/{field_device}/amp/{file_name}"
        # データを読み込み（TimeカラムをIndexに設定）
        df = pd.read_csv(file_path, index_col=0).set_index("Time", drop=True)

        # 信号処理を適用
        sp = SignalProcessor(df)
        ## 全て0の列を削除
        sp.remove_all_zero_col(inplace=True)
        ## hampelフィルターを適用
        sp.hampel_filter(inplace=True)
        ## 全て同じ値の列を削除
        sp.remove_all_value_col(inplace=True)
        ## 標準化
        sp.standardize(inplace=True)

        # データを保存
        Util.create_path(f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/")
        sp.get_data().to_csv(f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/{file_name}", index=True)